### Your lab resolution :

# Challenge 1: Fork Languages

In [80]:
import requests
import pandas as pd

In [81]:
url = 'https://api.github.com/repos/ironhack-datalabs/bcn-feb-2019/forks'
response = requests.get(url)
response.status_code

200

In [82]:
languages = set()
forks = response.json()

for fork in forks:
    language_response = requests.get(fork['languages_url'])
    for key in list(language_response.json().keys()):
        languages.add(key)
print(languages)




{'Python', 'HTML', 'Jupyter Notebook'}


# Challenge 2: Count Commits

In [83]:
url = 'https://api.github.com/repos/ta-data-pt-rmt/lab-mysql-first-queries/pulls'
response = requests.get(url)
response.status_code

200

In [84]:
from datetime import datetime, timedelta
pulls = response.json()
count = 0
for pull in pulls:
    commit_response = (requests.get(pull['commits_url'])).json()
    for commit in commit_response:
        date_string = commit['commit']['author']['date']
        given_date = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%SZ")
        # Calculate the date 3 weeks ago from the current date
        three_weeks_ago = datetime.utcnow() - timedelta(weeks=3)
        # Compare the given date with the date 3 weeks ago
        if given_date > three_weeks_ago:
            count += 1
print(count)



10
